In [2]:
#!pip3 install import_ipynb
#!pip3 install mpl_finance

import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates 
from dateutil.relativedelta import relativedelta
import os 
import pandas_datareader as web
import matplotlib as mpl
import pylab

In [3]:
def get_data(ticker, verbose = False):
    
    '''
    Collects open high low close ,adj close and volume from yahoo
    API and saves it into a folder `stock_dfs` as a csv file
    
    Parameters
    ----------
    ticker: stock ticker
        to look up a stock on yahoo api 
    verbose: boolean 
        if True, it just outputs first few terms for me to 
        check. keep it False for most usage

    Returns
    -------
    ret : dataframe
        returns a dataframe of ticker and also saves it in 
        folder `stock_dfs` '''
    
    if not os.path.exists('stock_dfs'):  #if folder doesn't exists, this creates it
            os.makedirs('stock_dfs')

    end = dt.datetime.today()
    end = dt.date(end.year, end.month, end.day)   #to get todays date
    #end = end.date();
    #end = dt.datetime(2019,2,9)
    start = end - relativedelta(years = 10, months = 0, days = 0)  #only looks at last 10 years

    if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
        try:
            df = web.DataReader(ticker,'yahoo',start,end);
            print('Collecting {} from Yahoo API.... \n'.format(ticker));
            df.to_csv('stock_dfs/{}.csv'.format(ticker));
            
            if verbose:
                print(ticker)
                print(df.tail())
            else:
                return df
            
        except: 
            print("The ticker '{}' does not exist in yahoo API. Please go on https://uk.finance.yahoo.com/ to find your ticker".format(ticker))
    else:
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker),
                         parse_dates=True,index_col=0)
        if __name__=="__main__":
            print('{} data already saved in stock_dfs. Building graph....'.format(ticker))
            print('')
        if verbose:
            print('Already have {} in stock folder \n'.format(ticker));
            print(ticker)
            print(df.tail())

        else:
            return df

'''Very useful when you import this file to other py file 
and you do not want to print this'''

if __name__=="__main__":
    print(get_data('TWTR',verbose = True))

TWTR data already saved in stock_dfs. Building graph....

Already have TWTR in stock folder 

TWTR
                 High        Low       Open      Close    Volume  Adj Close  \
Date                                                                          
2019-09-09  45.860001  43.840000  45.599998  44.259998  14547400  44.259998   
2019-09-10  43.950001  42.509998  43.900002  43.250000  14040100  43.250000   
2019-09-11  43.779999  43.049999  43.549999  43.250000   7861200  43.250000   
2019-09-12  44.110001  43.110001  43.759998  43.200001   8225300  43.200001   
2019-09-13  43.189999  42.799999  43.150002  42.799999   3041461  42.799999   

                 252D        42D  42D-252D        RSI      MACD  MACD_EMA9  \
Date                                                                         
2019-09-09  30.932604  36.374619  5.442014  60.491535  2.570765   3.571630   
2019-09-10  30.929500  36.442776  5.513275  55.240043  2.384405   3.644280   
2019-09-11  30.929188  36.546833  5

In [4]:
def specified_prices(data,years = 0, months = 3, days = 0):
  ''' 
  For the data we have for the stock, this function allows 
  to filter the data in the given period specified by the user 
  
  Parameters
    ----------
    data: dataframe
        make sure the index of this dataframe is dates or else
        you may get an error 
    years, months, days: integer
        to give a specified timeframe for plots and use for other 
        functions

   Returns
   -------
   ret : dataframe
        returns a dataframe of ticker which is cut to a given time-period 
  '''
  
  end = data.index[-1]  #make sure the data index is date and takes the most recent date
  start = end - pd.Timedelta(years = years, months = months, days = days,freq='B')
  data = data[start:]
  
  return data 

if __name__=="__main__":
    data = get_data('TWTR',verbose = False)
    print(specified_prices(data, months = 2).head())
    #print(help(specified_prices))

TWTR data already saved in stock_dfs. Building graph....



TypeError: Invalid type <class 'str'>. Must be int or float.

In [4]:
def simple_moving_average(data,window = 10):
    mean = data.rolling(window).mean()
    return mean

def exponential_moving_average(data,window = 10):
    mean = data.ewm(span=window, adjust=False).mean()
    mean = mean.shift(window-1)
    return mean

#data = pd.read_csv('stock_dfs/AAPL.csv',parse_dates=True,index_col=0)['Adj Close']
#df = exponential_moving_average(data,window = 10).values 


In [7]:
def add_col(data,ticker,heading = 'new_col'):
    ''' 
  For the data we have for the stock, this function allows 
  to add a column to the ticker.csv file 
  
  Parameters
    ----------
    data: dataframe
        to add to the df if it doesn't exist
    ticker: string
        for the stock
    heading: string
        for the title of the column
        
   Returns
   -------
   ret : dataframe
        returns a saved copy of the updated csv in stock_dfs 
        with the new column. 
  '''
    if os.path.exists('stock_dfs/{}.csv'.format(ticker)):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker),
                         parse_dates=True,index_col=0)
        
        df[heading] = data.values
        df.to_csv('stock_dfs/{}.csv'.format(ticker));
        
    else: 
        print("The csv file {}.csv doesn't exist")
        
#add_col('GOOG')

In [6]:
'''def candle_rsi(ticker,window = 10, years = 0, months = 3, days = 0,rsi_n = 14, verbose = True):
 """
    Plot 3 graphs in the following order: 
    
    - Plot the RSI whose function is in rsi.ipynb whose value ranges from
    0 to 100. Gives the users an indicator to predict stock price movement
    
    - Plot the time, open, high, low, close as a vertical line ranging
    from low to high.  Use a rectangular bar to represent the
    open-close span.  If close >= open, use colorup to color the bar,
    otherwise use colordown
    
    - On the above chart simple and exponential moving averages are 
    drawn for default window being 10 days
    
    - Volume graph is overlayed on top of candle_stick graph
    
    Parameters
    ----------
    ticker: stock ticker
        to look up a stock on yahoo api
    window: integer 
        the timeframe for exponential_moving_average() which is set to 10
        as a default
    years, months, days: integer
        for the use of specified_prices() to give a specified timeframe 
        for plots 
    rsi_n: integer 
        the timeframe for rs.rsiFunc()
    verbose: boolean 
        if my years>10, then if False, the graph will print everything it has
        for the prices.

    Returns
    -------
    ret : graph
        returns a graph and also saves the image in `price_plots` folder
    """  
    
 if years>10 and verbose: 
    print('In get_data() method, I have only extracted data for the last 10 years, please change the above method for more historic data\nIf you would like to see all the prices plot, then make verbose = False')
 
   ######## sorting out the data so that code is efficient and doesn't reload when not necessary ################  
 else:
    if os.path.exists('stock_dfs/{}.csv'.format(ticker)):
        data = pd.read_csv('stock_dfs/{}.csv'.format(ticker),
                     parse_dates=True,index_col=0)
        
        if not 'RSI' in data.columns: 
            rs.rsiFunc(ticker,n=rsi_n);
            data = pd.read_csv('stock_dfs/{}.csv'.format(ticker),
                 parse_dates=True,index_col=0)
    else: 
        rs.rsiFunc(ticker,n=rsi_n);
        data = pd.read_csv('stock_dfs/{}.csv'.format(ticker),
                 parse_dates=True,index_col=0)
        
    ###################
    
    ############ filtering data for timeperiod specified by the user ####################################
    df = specified_prices(data,years=years,months=months,days=days)   
    
    close = specified_prices(df['Adj Close'],years=years,months=months,days=days)
    volume = specified_prices(df['Volume'],years=years,months=months,days=days)
    rsi = specified_prices(df['RSI'],years=years,months=months,days=days)
    
    sma = simple_moving_average(close,window = window)
    ema = exponential_moving_average(close, window = window)
    SP = np.count_nonzero(~np.isnan(ema.values))  #counting number of non NaN rows
    date = ema.index[window-1]  #getting date of first non - NaN number
    
    df_ohlc = close.resample('10D').ohlc()[date:]  #only looking for data starting after ema line
    df_ohlc.reset_index(inplace=True)
    df_ohlc['Date'] = df_ohlc['Date'].map(mdates.date2num)
    ###########################
    
    ########### Plotting begins ####################################
    mpl.rcParams.update(mpl.rcParamsDefault)  #to remove plt.style.use and have default setting
    mpl.rcParams.update({'font.size':9})  #specifying the font-size 

    
    fig = plt.figure(figsize=(13,10),facecolor='#07000d') 
    #plt.style.use('ggplot')
    #plt.style.use('dark_background')
    
    ############# so here I plotting the candle_stick and SMA/EMA #######################
    ax1 = plt.subplot2grid((5,1),(1,0),rowspan = 4, colspan = 1,facecolor='#07000d')

    candlestick_ohlc(ax1,df_ohlc.values,width=7,colorup='#27d817',colordown = '#ff1717',alpha = 0.7);
    
    # plotting SMA and EMA 
    ax1.plot(df.index[-SP:],sma[-SP:],label = 'SMA {}D'.format(window),linewidth = 1.4)
    ax1.plot(df.index[-SP:],ema[-SP:],label = 'EMA {}D'.format(window),linewidth = 1.4)
    
    maLeg = ax1.legend(loc = 9, ncol = 2, prop={'size':7},fancybox = True)  #loc = 3, prop={'size':7},fancybox = True
    maLeg.get_frame().set_alpha(0.4) #set the background of legend to shade
    textEd = pylab.gca().get_legend().get_texts()
    pylab.setp(textEd[0:5],color = 'w') #changing the colour of legend to white
    
    ax1.set_ylabel("Stock Price and Volume",color = 'w');
    ax1.set_xlabel("Date",color = 'w')
    
    ax1.tick_params(axis='y', colors='w')
    ax1.tick_params(axis='x', colors='w')
    
    plt.xticks(rotation=20);
    
    ax1.grid(True, color='w', linestyle='--', linewidth=0.8,alpha = 0.3)
    
    ax1.spines['bottom'].set_color("#5998ff")
    ax1.spines['top'].set_color("#5998ff")
    ax1.spines['left'].set_color("#5998ff")
    ax1.spines['right'].set_color("#5998ff")
    
    ################# Here I am plotting the rsiFunc() graph on the top ########################
    ax0 = plt.subplot2grid((5,1),(0,0),rowspan = 1, colspan = 1, sharex = ax1,facecolor = '#07000d')
    
    rsicolor = '#00ffe8'
    ax0.plot(rsi.index[-SP:],rsi[-SP:],label = 'RSI {}D'.format(rsi_n),color = rsicolor,linewidth = 1.5)
    ax0.axhline(y=70, color = rsicolor,linestyle = '-');
    ax0.axhline(y=30, color = rsicolor,linestyle = '-');
    
    ax0.fill_between(rsi.index[-SP:],rsi[-SP:],70,where=(rsi[-SP:]>=70),facecolor = rsicolor,edgecolor=rsicolor)
    ax0.fill_between(rsi.index[-SP:],rsi[-SP:],30,where=(rsi[-SP:]<=30),facecolor = rsicolor,edgecolor=rsicolor)
    
    plt.setp(ax0.get_xticklabels(),visible = False)  #remove xaxis of ax0
        
    ax0.set_ylabel("RSI",color= 'w')
    ax0.tick_params(axis='y', colors='w')
    ax0.set_yticks([30,70])
    
    ax0.spines['bottom'].set_color("#5998ff")
    ax0.spines['top'].set_color("#5998ff")
    ax0.spines['left'].set_color("#5998ff")
    ax0.spines['right'].set_color("#5998ff")
    
    ############ To plot volume graph on top of the candle_stick diagram ###############
    volumeMin = 0  #volume.min()

    ax1v = ax1.twinx()
    ax1v.plot(volume.index[-SP:],volume[-SP:],label = 'Volume',color = '#00ffe8',linewidth = 1.4,alpha = 0.6)
    ax1v.fill_between(volume.index[-SP:],volumeMin, volume[-SP:], facecolor = '#00ffe8',alpha=0.4,label = 'Volume')
    
    #let's you adjust the stock prices ticks for x-axis for the dates [start,end,step]
    ax1v.xaxis.set_ticks(volume.index[np.arange(len(volume)-SP, len(volume), int(len(volume)/20))]) 
    ax1v.xaxis.set_major_formatter(mdates.DateFormatter("%m/%Y"))

    ax1v.set_ylim(0,6*volume.max())  #shrinks the size of volume on the graph
   
    ax1v.grid(False)
    
    ax1v.spines['bottom'].set_color("#5998ff")
    ax1v.spines['top'].set_color("#5998ff")
    ax1v.spines['left'].set_color("#5998ff")
    ax1v.spines['right'].set_color("#5998ff")
    
    #ax1v.legend(loc = 4)  #loc = 3, prop={'size':7},fancybox = True
    #######################################################
    
    ##### title for the graph
    plt.suptitle('{}'.format(ticker,window),color = 'w')
   
    ###### to give a graph a tighter fit ######################################
    fig.tight_layout()
    fig.subplots_adjust(left = 0.1,right = 0.93,top = 0.95,bottom = 0.14 ,wspace = 0.2,hspace=0.0) 

    plt.show()  #comment this to not give an output but it will still save image 
    
    ################ to create a folder (if doesn't exist) and save images in this folder
    if not os.path.exists('price_plots'):  #if folder doesn't exists, this creates it
            os.makedirs('price_plots')
    fig.savefig('price_plots/{}({}y,{}m,{}d).png'.format(ticker,years,months,days),facecolor = fig.get_facecolor(),format='png')  #dpi=1000
  
if __name__=='__main__':
    candle_rsi('AAPL',window = 10, years = 2, months = 0, days = 0,rsi_n = 14,verbose = True)  ''';
    
#os.rmdir('Untitled Folder')
#os.remove('chi_osc.ipynb')

In [7]:
'''def candle_moving(ticker,window = 10, years = 0, months = 3, days = 0, verbose = True):
  if years>10 and verbose: 
    print('In get_data() method, I have only extracted data for the last 10 years, please change the above method for more historic data\nIf you would like to see all the prices plot, then make verbose = False')
   
  else:
    df = specified_prices(get_data(ticker),years=years,months=months,days=days)
    close = specified_prices(df['Adj Close'],years=years,months=months,days=days)
    volume = specified_prices(df['Volume'],years=years,months=months,days=days)

    df_ohlc = close.resample('10D').ohlc()
    df_ohlc.reset_index(inplace=True)
    df_ohlc['Date'] = df_ohlc['Date'].map(mdates.date2num)

    sma = simple_moving_average(close,window = window)
    ema = exponential_moving_average(close, window = window)
    
    mpl.rcParams.update(mpl.rcParamsDefault)  #to remove plt.style.use
    mpl.rcParams.update({'font.size':9})

    
    fig = plt.figure(figsize=(13,10),facecolor='#07000d') #
    #plt.style.use('ggplot')
    #plt.style.use('dark_background')
    ax1 = plt.subplot2grid((5,1),(0,0),rowspan = 4, colspan = 1,facecolor='#07000d')

    candlestick_ohlc(ax1,df_ohlc.values,width=7,colorup='#27d817',colordown = '#ff1717',alpha = 0.8);
    ax1.plot(df.index,sma,label = 'SMA {}D'.format(window),linewidth = 1.4)
    ax1.plot(df.index,ema,label = 'EMA {}D'.format(window),linewidth = 1.4)
    ax1.legend(fancybox = True)  #loc = 3, prop={'size':7},fancybox = True
    
    ax1.set_ylabel("Stock Price");
    
    ax1.grid(True, color='w', linestyle='--', linewidth=0.8,alpha = 0.3)
    
    ax1.yaxis.label.set_color("w")
    ax1.spines['bottom'].set_color("#5998ff")
    ax1.spines['top'].set_color("#5998ff")
    ax1.spines['left'].set_color("#5998ff")
    ax1.spines['right'].set_color("#5998ff")
    ax1.tick_params(axis='y', colors='w')
    #ax1.yaxis.set_ticks(df_ohlc[np.arange(0, len(df_ohlc), int(len(df_ohlc)/20))])
    plt.xticks(rotation=20);
    
    ax2 = plt.subplot2grid((5,1),(4,0),rowspan = 1, colspan = 1, sharex = ax1,facecolor = '#07000d')
    
    volumeMin = 0  #volume.min()
    
    ax2.plot(volume.index,volume,label = 'Volume',color = '#00ffe8',linewidth = 1.4)
    ax2.fill_between(volume.index,volumeMin, volume, facecolor = '#00ffe8',alpha=0.5)
    
    plt.xlabel("Date",color= 'w')
    #let's you adjust the stock prices ticks for x-axis for the dates [start,end,step]
    ax2.xaxis.set_ticks(volume.index[np.arange(0, len(volume), int(len(volume)/10))]) 
    ax2.xaxis.set_major_formatter(mdates.DateFormatter("%m/%Y")) 
    ax2.set_ylabel('Volume')
    
    ax2.grid(False)
    ax2.yaxis.label.set_color("w")
    ax2.spines['bottom'].set_color("#5998ff")
    ax2.spines['top'].set_color("#5998ff")
    ax2.spines['left'].set_color("#5998ff")
    ax2.spines['right'].set_color("#5998ff")
    ax2.tick_params(axis='y', colors='w')
    ax2.tick_params(axis='x', colors='w')
    
    plt.suptitle('{} Candle Stick and {}D SMA and EMA'.format(ticker,window),color = 'w')
   
    fig.tight_layout()
    fig.subplots_adjust(left = 0.1,right = 0.93,top = 0.95,bottom = 0.14 ,wspace = 0.2,hspace=0.0) 

    plt.show()
    
    if not os.path.exists('price_plots'):  #if folder doesn't exists, this creates it
            os.makedirs('price_plots')
    fig.savefig('price_plots/{}({}y,{}m,{}d).png'.format(ticker,years,months,days),facecolor = fig.get_facecolor(),format='png')  #dpi=1000
  
if __name__=='__main__':
    candle_moving('AAPL',window = 10, years = 1, months = 10, days = 0,verbose = True)  ''';
    
#os.rmdir('Untitled Folder')
#os.remove('chi_osc.ipynb')

In [8]:
''' def candle_moving(ticker,window = 10, years = 0, months = 3, days = 0, verbose = True):
  if years>10 and verbose: 
    print('In get_data() method, I have only extracted data for the last 10 years, please change the above method for more historic data\nIf you would like to see all the prices plot, then make verbose = False')
   
  else:
    df = specified_prices(get_data(ticker),years=years,months=months,days=days)
    data = specified_prices(df['Adj Close'],years=years,months=months,days=days)

    df_ohlc = data.resample('10D').ohlc()
    df_ohlc.reset_index(inplace=True)
    df_ohlc['Date'] = df_ohlc['Date'].map(mdates.date2num)

    sma = simple_moving_average(data,window = window)
    ema = exponential_moving_average(data, window = window)
    
    fig = plt.figure(figsize=(15,10),facecolor='#07000d')
    plt.style.use('ggplot')
    plt.style.use('dark_background')
    ax1 = plt.subplot2grid((6,1),(0,0),rowspan = 6, colspan = 1)

    candlestick_ohlc(ax1,df_ohlc.values,width=7,colorup='#27d817',colordown = '#ff1717',alpha = 0.8);
    ax1.xaxis.set_major_formatter(mdates.DateFormatter("%m/%Y"))   
    plt.xticks(rotation=30);
    plt.xlabel("Date");
    plt.ylabel("Stock Price");
    ax1.xaxis_date()

    plt.plot(df.index,sma,label = 'SMA {}D'.format(window),linewidth = 1.4)
    plt.plot(df.index,ema,label = 'EMA {}D'.format(window),linewidth = 1.4)
    plt.legend()
    
    plt.title('{} Candle Stick and {}D Sma and Ema'.format(ticker,window))
    
    #fig.tight_layout()
    fig.subplots_adjust(hspace=0.1) 

    plt.show()
  
if __name__=='__main__':
    candle_moving('GOOG',window = 10, years = 1, months = 0, days = 0,verbose = True)  ''';

In [9]:
'''Playing around with stuff


df = get_data('AAPL')
data = df['Adj Close']


sma = simple_moving_average(data)
ema = exponential_moving_average(data)
data.head()
sma.tail()
ema.tail()


df_ohlc = data.resample('10D').ohlc()
df_ohlc.tail()
df_ohlc.reset_index(inplace=True)
df_ohlc['Date'] = df_ohlc['Date'].map(mdates.date2num)

plt.style.use('ggplot')
ax1 = plt.subplot2grid((6,1),(0,0),rowspan = 6, colspan = 1)
#ax2 = plt.subplot2grid((6,1),(0,0),rowspan = 6, colspan = 1,sharex = ax1, sharey = ax1)
#ax3 = plt.subplot2grid((6,1),(0,0),rowspan = 6, colspan = 1,sharex = ax1, sharey = ax1)

candlestick_ohlc(ax1,df_ohlc.values,width=7,colorup='g');
ax1.xaxis_date()

plt.plot(df.index,sma,label = 'SMA')
plt.plot(df.index,ema,label = 'EMA')
plt.legend()
plt.show() ''';